In [2]:
import torch
from torch import nn 
class NeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 30),
            nn.ReLU(),

            nn.Linear(30, 20),
            nn.ReLU(),

            # output layer
            nn.Linear(20, num_outputs)            
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [3]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [4]:
# check the number of trainable parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("total number of trainable model parameters:", num_params)

total number of trainable model parameters: 2213


In [5]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [6]:
# with grad
torch.manual_seed(123)
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1486,  0.1065, -0.1208]], grad_fn=<AddmmBackward0>)


In [9]:
# without grad, we have to call softmax function manually
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)


tensor([[0.3013, 0.3889, 0.3098]])


## Setting up efficient data loaders (Appendix A.6)

In [10]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [16]:
from torch.utils.data import Dataset 

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    def __len__(self):
        return self.labels.shape[0]
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [17]:
from torch.utils.data import DataLoader 

torch.manual_seed(123)


# Instantiating data loader
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [18]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [23]:
# we drop the last batch in each epoch, because the last batch only have a single example, 
# this could disturb convergence
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    # num_workers=0 to make the data loading happen in the main process, not separate worker process.
    # this is ok if the dataset is small
    num_workers=0,
    drop_last=True
)
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [ 2.7000, -1.5000]]) tensor([1, 1])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.9000,  2.9000]]) tensor([0, 0])


In [29]:
import torch.nn.functional as F 

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
# SGD = stochastic gradient descent
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}",
              f"| Batch {batch_idx:03d}/{len(train_loader):03d}",
            f"| Train Loss {loss:.2f}")

Epoch: 001/003 | Batch 000/002 | Train Loss 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss 0.00


In [33]:
torch.set_printoptions(sci_mode=False)
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])
tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [36]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [37]:
predictions == y_train

tensor([True, True, True, True, True])

In [ ]:
def compute